In [3]:
#import the necessary modules/libraries
import os
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss, r2_score
from tqdm import tqdm_notebook
import time
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F  
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import csv

In [4]:
#Read the experimental dataset

In [5]:
df=pd.read_csv('ExpDataMatrix.csv') # reading the only experimental dataset

In [6]:
df.shape #check the dataset shape; 258 reactions and 137 features with one output column

(258, 138)

In [7]:
#Build your model
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        self.regressor = nn.Sequential(
            nn.Linear(137, 150),
            nn.ReLU(),
            nn.Linear(150, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
            
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

In [9]:
def expDataSet(seed, df1):
    """Given a seed value and the dataset it returns 
    seed value, train, validation and test rmses"""
    exp_feat=df1.iloc[:, :-1]
    exp_ee=df1.iloc[:, -1]
    X_exp_train, X_test, y_exp_train, y_test = train_test_split(exp_feat, exp_ee, test_size=0.2, random_state=seed)
    #test set is ready; i.e. X_test and y_test
    ind_val=X_test.index #just noting the index of the test set for future
    # further divide the training set into train and validation set
    X_train, X_val, y_train, y_val=train_test_split(X_exp_train, y_exp_train, test_size=0.2, random_state=0)
    #convert all of them to tensors
    X_train=torch.FloatTensor(X_train.values)
    X_val=torch.FloatTensor(X_val.values)
    y_train=torch.FloatTensor(y_train.values)
    y_val=torch.FloatTensor(y_val.values)
    X_test=torch.FloatTensor(X_test.values)
    y_test=torch.FloatTensor(y_test.values)
    #Instantiate the model
    model=NN()
    criterion=torch.nn.MSELoss()
    optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
    model.train()
    epoch=1000
    loss_arr=[]
    loss_val_arr=[]
    for epoch in range(epoch):
        optimizer.zero_grad()
        #Forward pass
        y_pred=model(X_train)
        #compute loss
        loss=criterion(y_pred.squeeze(), y_train)
        loss_arr.append(loss.item())
        #print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
        #Backward pass
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            model.eval()
            y_pred_val=model(X_val) #feed the X_val to the model to predict
            loss_val=criterion(y_pred_val.squeeze(), y_val) #calculate mse for validation set
            loss_val_arr.append(loss_val.item())
            #print('Epoch {}: val loss: {}'.format(epoch, loss_val.item()))
    
    #Now evaluate the test set
    model.eval()
    y_pred = model(X_test) #feed the X_test to model to predict output
    after_train = criterion(y_pred.squeeze(), y_test) #calculate test mse
    test_rmse=sqrt(after_train) #calculate test rmse
    
    y_pred=y_pred.detach().numpy()
    y_pred=y_pred.reshape(-1)
    #dfOutput=pd.DataFrame(zip(ind_val, y_test.numpy(), y_pred), columns=['ind', 'y_test', 'y_pred'])
    #dfOutput.to_csv('predOutputSeed.csv', index=False) #to save the result
    return seed, sqrt(loss_arr[-1]), sqrt(loss_val_arr[-1]), test_rmse 

In [10]:
#run it for hundred different seed values
result=[]
for i in range(0, 100):
    result.append(expDataSet(i,df)) 

In [11]:
#save the result into a dataframe 
dfresult=pd.DataFrame(result, columns=['seed', 'train_rmse', 'val_rmse', 'test_rmse'])

In [ ]:
#to save the dataframe into a csv file
#dfresult.to_csv('dnnResultExpData.csv', index=False)

In [12]:
dfresult.describe() 
#Hence the average test RMSE over 100 run = 10.72+/-3.04
#train rmse=9.20+/-1.43

,seed,train_rmse,val_rmse,test_rmse
count,100.000000,100.000000,100.000000,100.000000
mean,49.500000,9.201701,10.385358,10.728745
std,29.011492,1.436256,3.362785,3.041826
min,0.000000,5.846301,5.824433,6.241481
25%,24.750000,7.937773,7.942764,8.372097
50%,49.500000,9.177128,9.173895,9.833169
75%,74.250000,10.524605,13.631752,12.878993
max,99.000000,11.949717,19.715849,18.686283
